# UPPP 214 - Week 4

In [ ]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from fsspec import filesystem

<a target="_blank" href="https://colab.research.google.com/github/knaaptime/uppp214-winter26-assn/blob/main/week4/relationships.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Data Preparation

For class assignments and exercises, I generally give you a good dataset to start with. But it can also be very instructive to see how we go from zero to 'tidy data'.

We start by reading in three datasets:

1. traffic collisions in CA
2. census demographic information
3. environmental data from EJSCREEN

the first is a set of points, the latter two are *blockgroup* polygons

In [ ]:
fs = filesystem("https")

collisions = gpd.read_parquet("https://github.com/oturns/example_datasets/raw/refs/heads/main/collisions/ca_collisions.pq", filesystem=fs)
bgs = gpd.read_parquet("https://github.com/oturns/example_datasets/raw/refs/heads/main/acs/ca_blockgroups_2021.pq", filesystem=fs)
environment = gpd.read_parquet("https://github.com/oturns/example_datasets/raw/refs/heads/main/environment/ca_ejscren_2021.pq", filesystem=fs)

the Los Angeles metropolitan region contains two counties: LA and Orange. Their FIPS codes are `06037` and `06059`, respectively

In [ ]:
la_msa_fips = ['06037', '06059']

we can use those two counties as a filter. As we know, FIPS codes are hierarchical; so if we want all blockgroups in LA and OC, then we want all those whose first 5 characters of the FIPS code (called `geoid`) are either 06059 or 06037. We can select those using "string methods"

In [ ]:
bgs = bgs[bgs.geoid.str[:5].isin(la_msa_fips)]

In [ ]:
bgs.head()

In [ ]:
bgs['county'] = bgs.geoid.str[:5]

In [ ]:
bgs.plot()

lets consider only places where there is non-zero population

In [ ]:
bgs = bgs[bgs['n_total_pop']>0]

In [ ]:
bgs.plot()

the `environment` dataset has data from the EPA environmental justice screening tool. Most useful for our purposes are the estiamtes of different chemical emissions like Ozone and PM2.5. Since both datasets are at the blockgroup level and both have FIPS codes, we can use that information to merge them together

In [ ]:
environment.head()

See [here](https://open.quiltdata.com/b/spatial-ucr/tree/epa/ejscreen/) for a list of columns and what they mean

pandas `merge` function combines two dataframes based on their share values in a given column (if you're from SQL or database world, this is a *table join*). Since we already dropped blockgroups from `bgs` that have non-zero population, we want to use a "left-join", which means we only retain rows if they are in the left-hand dataframe (bgs)

In [ ]:
df = bgs.merge(environment, on='geoid', how='left')

now the dataframe `df` has information from both Census and EPA

In [ ]:
df.shape

In [ ]:
df.columns

remember we can look at a empirical distribution of single column via the `plot` or `hist` methods

In [ ]:
df['median_household_income'].hist()

In [ ]:
df['median_household_income'].plot(kind='hist')

In [ ]:
df['median_household_income'].plot(kind='hist', bins=25)

In [ ]:
df['median_household_income'].plot(kind='density')

income usually follows a log-normal distribution, so if we take the logarithm, we get something very close to a Normal distribution

...except that it's top-coded at $250,000

In [ ]:
df['median_household_income'].plot(kind='box')

In [ ]:
df.plot?

In [ ]:
df['median_household_income'].apply(np.log).plot(kind='hist', bins=35)

In [ ]:
df['median_household_income'].plot(kind='hist', logx=True, bins=35)

In [ ]:
df['median_household_income'].plot(kind='hist', bins=35, title='Histogram of Median Household Income', xlabel='Median Home Value')

We can also create plots using `seaborn`

In [ ]:
sns.histplot?

In [ ]:
sns.histplot(data=df, x='median_home_value', bins=20)

With seaborn we can easily combine the density and histogram plots into one to see how the continuous kernel density estimate falls over the bins

In [ ]:
sns.histplot(data=df, x='median_home_value', bins=20, kde=True, color='red', fill=True)

In [ ]:
sns.boxplot(data=df, x='median_home_value')

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
sns.boxplot(data=df, x='median_home_value', ax=ax)

ax.set_title('Boxplot of Median Home Values in LA Metro')
ax.set_xlabel('Median Home Values')


In [ ]:
sns.boxplot(data=df, y='median_home_value', x='county')

In [ ]:
sns.violinplot?

In [ ]:
sns.violinplot(data=df, x='median_home_value',)

In [ ]:
sns.violinplot(data=df, y='median_home_value', x='county')

In [ ]:
sns.violinplot(data=df, y='p_edu_college_greater', x='county')

seaborn also has multiple "styles" preconfigured for different aesthethics

In [ ]:
sns.set_style?

In [ ]:
for style in ['darkgrid', 'whitegrid', 'dark', 'white', 'ticks']:
    sns.set_style(style)
    ax = sns.violinplot(data=df, y='p_edu_college_greater', x='county',)
    ax.set_title(style)
    plt.show()

Now we can look at relationships between two or more variables

In [ ]:
df[['median_home_value', 'p_edu_college_greater']]

In [ ]:
df[['median_home_value', 'p_edu_college_greater']].corr()

In [ ]:
sns.scatterplot?

In [ ]:
sns.scatterplot(data=df, x='median_home_value', y='p_edu_college_greater')

In [ ]:
sns.scatterplot(data=df, x='median_home_value', y='p_edu_college_greater', alpha=0.2)

We can add a third 'visual variable', like color/hue, to differentiate between a (third) categorical variable. For instance we can look at the scatterplots by county

In [ ]:
sns.scatterplot(data=df, x='median_home_value', y='p_edu_college_greater', hue='county',alpha=0.2)

the `corr` method in the dataframe returns the pairwise correlation coefficient for each pair of inputs. The correlation between educational attainment and median home values is quite high at 0.69

In [ ]:
cols = ['median_home_value', 'p_edu_college_greater', 'median_household_income', 'p_edu_hs_less', 'p_married', 'p_vacant_housing_units', 'p_persons_over_60']

In [ ]:
df[cols]

In [ ]:
df[cols].corr()

To look at rank correlation, we specify `method='spearman'`

In [ ]:
df[cols].corr('spearman')

many of these variables have extreme values, so some of the correlations change a bit; the relationship between home values and income stays roughly the same. But the correlation between home values and education gets even stronger

the `corr` method gives back a correlation matrix, so one useful way of understanding correlation among multiple groups is via a [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html)

In [ ]:
sns.heatmap?

In [ ]:
sns.heatmap(df[cols].corr(), cmap='RdBu_r')

the default options are not perfect. To make this better we can specify the range ourselves. We know the correlation coefficient varies from -1 to 1, so lets set those as the max and min values, and place the value of the correlation coefficient in each cell

In [ ]:
sns.heatmap(df[cols].corr(), cmap='RdBu_r', vmin=-1, vmax=1, annot=True)

Another option is to use a [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) which gives pairwise scatterplots and a histogram along the diagonal

In [ ]:
sns.pairplot(df[cols])

### Demographics and the Environment

lets look at some examples together:

|var |description|
-----|------------|
| DSLPM	| Diesel particulate matter level in air|
| CANCER |	Air toxics cancer risk|
| RESP	| Air toxics respiratory hazard index|
| PTRAF	| Traffic proximity and volume|
| PWDIS	| Indicator for major direct dischargers to water|
| PNPL	| Proximity to National Priorities List (NPL) sites|
| PRMP	| Proximity to Risk Management Plan (RMP) facilities|
| PTSDF	| Proximity to Treatment Storage and Disposal (TSDF) facilities|
| OZONE	| Ozone level in air|
| PM25	| PM2.5 level in air|

### Collision Data

We can also look at some relationships between the datasets above and the prevalence of traffic accidents

In [ ]:
collisions.head()

In [ ]:
collisions = collisions[collisions["ACCIDENT_YEAR"] ==2023]

In [ ]:
collisions.shape

In [ ]:
collisions_by_bg = bgs.sjoin(collisions, how='left')

In [ ]:
collisions_by_bg

In [ ]:
collisions_by_bg = collisions_by_bg.groupby('geoid')['CASE_ID'].count()

In [ ]:
collisions_by_bg

In [ ]:
collisions_by_bg = collisions_by_bg.rename('collisions')

In [ ]:
collisions_by_bg

In [ ]:
df = df.merge(collisions_by_bg, left_on='geoid', right_index=True)

In [ ]:
df.head()

In [ ]:
df['collisions'].describe()

In [ ]:
df = df.set_geometry('geometry_x')

In [ ]:
df[['collisions', 'geometry_x']].explore('collisions', scheme='fisherjenks', style_kwds={'weight':0.5}, tiles='cartodb positron')

In [ ]:
df.collisions.hist()

In [ ]:
df.collisions.apply(np.log1p).hist()

In [ ]:
df[['n_total_pop', 'collisions']]

In [ ]:
df[['n_total_pop', 'collisions']].corr()

essentially 0 correlation between blockgroup popuolation and the number of collisions

In [ ]:
df['coll_per_capita'] = df['collisions'] / df['n_total_pop']

In [ ]:
df['coll_per_capita']

In [ ]:
df['coll_per_capita'].describe()

In [ ]:
df[['collisions', 'median_home_value']].corr()

In [ ]:
df[['coll_per_capita', 'median_home_value']].corr()

In [ ]:
df[['coll_per_capita', 'median_home_value']].corr()

In [ ]:
sns.scatterplot(data=df, y='collisions', x='median_home_value', alpha=0.5)

In [ ]:
sns.scatterplot(data=df, y='collisions', x='median_home_value')
sns.despine()

In [ ]:
df[['collisions', 'PTRAF']].corr()

In [ ]:
sns.scatterplot(data=df, x='PTRAF', y='collisions', alpha=0.2)

In [ ]:
sns.scatterplot(data=df, x='PTRAF', y='collisions', alpha=0.2, hue='county')

In [ ]:
# convert the coordinate system to UTM for accurate area measurement in meters
df = df.to_crs(df.estimate_utm_crs())

In [ ]:
df['sqm'] = df.area

In [ ]:
df[['sqm', 'collisions']].corr()

looking back at the map, we can intuit a bit of what's happening. There are few collisions in the very small polygons because there aren't many opportunities to collide; they are small enough that there's few cars in the polygon at any given time. But the largest polygons are in every rural areas, presumably with few roads or cars